In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import itertools
from collections import defaultdict

In [2]:
## my functions
from droom import chech_for_nan,check_for_duplicates,get_duplicates_indexes,get_all

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
deal_data = pd.read_csv('data_scientist_duplicate_detection.csv')

In [5]:
deal_data.shape

(15899, 13)

In [6]:
deal_data.tail(2)

,id,name,tagline,industry,industry_2,type,address,street,street_number,zip,country,city,sectors
15897,1435346,Horosh,NaN,Health,Dating,2,"Princeton Mews, 167 169 London Rd, Kingston up...",NaN,NaN,NaN,NaN,NaN,NaN
15898,1366367,Photon,Photon,Education,Kids,2,Poland,NaN,NaN,NaN,NaN,NaN,NaN


## Some comments on the Features
 * The city has some  so  not important cause the NaN can be replaced by Unknown, holds also for the country
 * particulary because both city  and country  are included in the address, also the adress
 * does not  contain steet name, number, zip for some countries or cities like for exmple 
 * United states, New York and we can't guess or do imputation here

In [7]:
chech_for_nan(deal_data)

There are NaN values in the dataset

Check Indvidual columns:

The column "id" is good
The column "name" is good
The column "tagline" has some NaN values
The column "industry" has some NaN values
The column "industry_2" has some NaN values
The column "type" is good
The column "address" is good
The column "street" has some NaN values
The column "street_number" has some NaN values
The column "zip" has some NaN values
The column "country" has some NaN values
The column "city" has some NaN values
The column "sectors" has some NaN values


As can be seen some columns are ok like the [id, name, type, address] which gives us some indications about the most import features. However, considering that here we are dealing with a supply chain problem therefore, apart from the id, name and the address of the distributing company, the street, street number as well as the zip code is of paramount considering the fact that in some cases they are not incuded in the address.

In [8]:
data=deal_data[deal_data['city'].isnull()]

In [9]:
data#['address']

,id,name,tagline,industry,industry_2,type,address,street,street_number,zip,country,city,sectors
15896,1344567,FlashhReal,Original augmented reality flash card app that...,Media,Kids,2,"Colombia, Bogota, D.C.",NaN,NaN,NaN,NaN,NaN,NaN
15897,1435346,Horosh,NaN,Health,Dating,2,"Princeton Mews, 167 169 London Rd, Kingston up...",NaN,NaN,NaN,NaN,NaN,NaN
15898,1366367,Photon,Photon,Education,Kids,2,Poland,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
deal_data[deal_data['name']=='Photon']

,id,name,tagline,industry,industry_2,type,address,street,street_number,zip,country,city,sectors
72,2944774,Photon,NaN,Kids,Education,2,"Bialystok, Woj. Podlaskie, Poland",NaN,NaN,15-007,Poland,Bialystok,NaN
15898,1366367,Photon,Photon,Education,Kids,2,Poland,NaN,NaN,NaN,NaN,NaN,NaN


As we can see that given that the city is Unknown, the address is known but it has some missing information for example, Poland, doesn't convey any message about where the company is located exactly. However, We know the name of the company and using that information, we can see that it appeared also in the dataset, but a company can have the same brand name within the same country or around the world, this also shows that the name cannot give us the full information needed, thus the address, the street, the streetnumber as well as the zip can help us identify where exactly the goods are coming from.

#### check for duplicates whole dataset and some individual columns

In [11]:
check_for_duplicates(deal_data)

There are duplicate rows in the dataset

Check Indvidual columns:

Column "id" has some duplicates
Column "name" has some duplicates
Column "tagline" has some duplicates
Column "industry" has some duplicates
Column "industry_2" has some duplicates
Column "type" has some duplicates
Column "address" has some duplicates
Column "street" has some duplicates
Column "street_number" has some duplicates
Column "zip" has some duplicates
Column "country" has some duplicates
Column "city" has some duplicates
Column "sectors" has some duplicates


#### # if the id was the only important feature here then  only one dublicate would be on this dataset

In [12]:
deal_data.loc[deal_data['id']==867000] 
#deal_data[deal_data['id'].duplicated()].shape

,id,name,tagline,industry,industry_2,type,address,street,street_number,zip,country,city,sectors
654,867000,Swarovski Group,Homepage - Swarovski Group,NaN,NaN,4,"Switzerland, Männedorf, Alte Landstrasse, 411",Alte Landstrasse,411,8708,Switzerland,Männedorf,NaN
655,867000,Swarovski Group,Homepage - Swarovski Group,NaN,NaN,4,"Switzerland, Männedorf, Alte Landstrasse, 411",Alte Landstrasse,411,8708,Switzerland,Männedorf,NaN


In [13]:
#deal_data['zip'].unique()

given that the 'id' is duplicated, the industries as well as the sectors are not known, showing that these features are not really important here also one can easily identify the type of industry as well as the sector by just looking at the goods. So, apart from the id, name, address, we also need the street, the street number as wel as the zip code. Moreover, the tagline plays a vital role since it contains probably a brief description of the goods so one can guess the industry as well as the sector. Given the address, one can also identify both the country and the city if included otherwise a zip code would be useful. 

###  select data when specific columns have null values

In [14]:
mask = deal_data['zip'].isnull() | deal_data['street_number'].isnull() | deal_data['street'].isnull() | deal_data['tagline'].isnull()

In [15]:
mask2 = deal_data['industry'].isnull() | deal_data['industry_2'].isnull() | deal_data['sectors'].isnull() 

In [16]:
masg = mask | mask2

In [17]:
data1 = deal_data[~masg]

In [18]:
 #| deal_data['sectors'].isnull() |  deal_data['city'].isnull() | 

In [19]:
data1.shape

(418, 13)

In [20]:
data2 = deal_data[masg]

In [21]:
deal_data.shape[0] == data1.shape[0] + data2.shape[0]

True

In [22]:
detect_duplicates = deal_data[~mask] # get the data where important features are known

In [23]:
data_with_selected_nan = deal_data[mask] # get the data where important features are not known

In [24]:
deal_data.shape[0] == detect_duplicates.shape[0] + data_with_selected_nan.shape[0] # Check if math ok. 😊 Great! 

True

In [25]:
check_for_duplicates(data_with_selected_nan)

The column 'id' doesn't have duplicates
Column "name" has some duplicates
Column "tagline" has some duplicates
Column "industry" has some duplicates
Column "industry_2" has some duplicates
Column "type" has some duplicates
Column "address" has some duplicates
Column "street" has some duplicates
Column "street_number" has some duplicates
Column "zip" has some duplicates
Column "country" has some duplicates
Column "city" has some duplicates
Column "sectors" has some duplicates


In [26]:
check_for_duplicates(detect_duplicates)

There are duplicate rows in the dataset

Check Indvidual columns:

Column "id" has some duplicates
Column "name" has some duplicates
Column "tagline" has some duplicates
Column "industry" has some duplicates
Column "industry_2" has some duplicates
Column "type" has some duplicates
Column "address" has some duplicates
Column "street" has some duplicates
Column "street_number" has some duplicates
Column "zip" has some duplicates
Column "country" has some duplicates
Column "city" has some duplicates
Column "sectors" has some duplicates


however, if the tagline is unknown and the city  is known, the dataset is not duplicated anymore showing that this feature is very important. The same holds when the streets,street_number, as well as the zip. Therefore, these features could be usefull for duplicate detection. So data when these features contain some null  values can be left untouched and only  use the other set when these features are known. Also considering this sub set, the id does not have duplicates in here since a company or good or service can be identified based on it. 

Also combining the cases we can see that the data is duplicated when either the city, sectors, industries are known and the other important features are known, the id has no dublicates, therefore, the city, sectors and industries are not important to detecting duplicates therefore we will use the detect_duplicates subset

## Analysis of subset data 

In [27]:
detect_duplicates['id'].duplicated().any()

True

### So to which features is the sub set data duplicated?

However,even though a tagline can help to get what the goods contain, it doesn't really  provide the necessary information for supply  chain. Also can be seen that if tagline is unknown, the city is known which deems the unnecessity of the tagline

In [28]:
get_duplicates_indexes(detect_duplicates,detect_duplicates.columns[0])#[0][-1][0]

{'value_counts': ({867000}, [2]), 'rows': [[99, 100]]}

##### uncomment when running from the top

In [29]:
duplicated = get_all(detect_duplicates)

In [30]:
duplicated['name'][0]['rows']

[[1468, 1504],
 [654, 1538],
 [24, 1822],
 [373, 2067],
 [613, 817],
 [1239, 1329],
 [1453, 1884],
 [904, 1432],
 [1855, 1861],
 [176, 1958],
 [84, 395],
 [730, 1910],
 [1601, 1997],
 [2044, 2104],
 [1836, 1988],
 [1002, 1065],
 [568, 1595],
 [673, 1813],
 [166, 1763],
 [99, 100],
 [523, 1230]]

In [31]:
#duplicated['tagline'][0]['rows'] # has NaN dublicated values so not important here

In [32]:
setA = set(list(itertools.chain.from_iterable(duplicated['address'][0]['rows'])))
setB = set(list(itertools.chain.from_iterable(duplicated['name'][0]['rows'])))
#setC = set(list(itertools.chain.from_iterable(duplicated['street_number'][0]['rows'])))
#setD = set(list(itertools.chain.from_iterable(duplicated['street'][0]['rows'])))
##setE = set(list(itertools.chain.from_iterable(duplicated['zip'][0]['rows'])))
setF = list(itertools.chain.from_iterable(duplicated['tagline'][0]['rows']))
setG = list(itertools.chain.from_iterable(duplicated['type'][0]['rows']))
#setH = set(list(itertools.chain.from_iterable(duplicated['industry'][0]['rows'])))
#setI = set(list(itertools.chain.from_iterable(duplicated['industry_2'][0]['rows'])))
#setJ = set(list(itertools.chain.from_iterable(duplicated['sectors'][0]['rows'])))
#setK = list(itertools.chain.from_iterable(duplicated['country'][0]['rows']))
#setL = list(itertools.chain.from_iterable(duplicated['city'][0]['rows']))
setM = list(itertools.chain.from_iterable(duplicated['id'][0]['rows']))

In [33]:
intersection = set.intersection(setA,
                                setB,
                                #setC,
                                #setD,
                                #setE,
                                setF,
                                setG,
                                #setH,
                                #setI,
                                #setJ,
                                #setK,
                                #setL,
                                setM
                                )

In [34]:
'industry','industry_2','sectors','country','city'

('industry', 'industry_2', 'sectors', 'country', 'city')

- By uncommenting the sets in order to find the point of intersection for all columns with duplicates, one can figure out which features contributed to the duplication. If we include inudustry_2 and sectors for example, there is no point of intersection at all. Including industry and the id features, one would would not find point of intesection. So the industry is not important. Repeating the procedure one would finally find the duplicated points which are a result of the ID;NAME;ADDRESS;TAGLINE and TYPE  

### ***** The ID of the dublicated data is 867000 which appeared twice. ****

In [35]:
duplicated['id']#[0]['rows']

[{'value_counts': ({867000}, [2]), 'rows': [[99, 100]]}]

In [36]:
list(intersection)

[99, 100]

In [37]:
deal_data[deal_data['id']==867000]

,id,name,tagline,industry,industry_2,type,address,street,street_number,zip,country,city,sectors
654,867000,Swarovski Group,Homepage - Swarovski Group,NaN,NaN,4,"Switzerland, Männedorf, Alte Landstrasse, 411",Alte Landstrasse,411,8708,Switzerland,Männedorf,NaN
655,867000,Swarovski Group,Homepage - Swarovski Group,NaN,NaN,4,"Switzerland, Männedorf, Alte Landstrasse, 411",Alte Landstrasse,411,8708,Switzerland,Männedorf,NaN


In [38]:
len(list(duplicated.keys()))

13

In [39]:
#list(duplicated.values())

#### **** This result can also be found using pandas duplicated function.

In [40]:
detect_duplicates.loc[detect_duplicates.duplicated(keep=False),:]

,id,name,tagline,industry,industry_2,type,address,street,street_number,zip,country,city,sectors
654,867000,Swarovski Group,Homepage - Swarovski Group,NaN,NaN,4,"Switzerland, Männedorf, Alte Landstrasse, 411",Alte Landstrasse,411,8708,Switzerland,Männedorf,NaN
655,867000,Swarovski Group,Homepage - Swarovski Group,NaN,NaN,4,"Switzerland, Männedorf, Alte Landstrasse, 411",Alte Landstrasse,411,8708,Switzerland,Männedorf,NaN


#### Save the results to csv file

In [41]:
final_data = deal_data[deal_data['id']==867000]

In [42]:
final_data = final_data.drop(['city','country','sectors','industry_2',
                              'industry','zip','street','street_number'],axis=1) 

In [43]:
final_data.to_csv('deal_room_intern_casestudy_dublicates.csv', index=False)

In [44]:
duplicated_data = pd.read_csv('deal_room_intern_casestudy_dublicates.csv')

In [45]:
duplicated_data

,id,name,tagline,type,address
0,867000,Swarovski Group,Homepage - Swarovski Group,4,"Switzerland, Männedorf, Alte Landstrasse, 411"
1,867000,Swarovski Group,Homepage - Swarovski Group,4,"Switzerland, Männedorf, Alte Landstrasse, 411"


In [46]:
#### I did not deal  with NaN because I was going to set them to  Unknown or not specified.